In [4]:
import random
numSeed = 42
random.seed(42)
from time import time
import numpy as np
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics

import sys  
sys.path.append('../')
from util.util import *
from loadFrey import *

## Load the datasets

In [7]:
xTrain, xTest = load_frey_face_dataset()
print(xTrain.shape)
print(xTest.shape)

(1400, 28, 20)
(565, 28, 20)


In [5]:
xTrain, xTest = load_frey_face_dataset()
xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
numTrain = len(xTrain)
numTest = len(xTest)
dimInput = np.prod(xTrain.shape[1:])
sizeDigit = xTrain.shape[1]

xTrain = xTrain.reshape((numTrain, dimInput))
xTest = xTest.reshape((numTest, dimInput))
print(xTrain.shape)
print(xTest.shape)

(1400, 560)
(565, 560)


## Declare the parameters and layers

In [ ]:
numEpochs = 50
sizeBatch = 32
dimInter = 64
dimEncode = 2
stdEps = 1.0 
ratRecon = 0.995
nameOptim = 'adam'

## Construct the encoder and decoder

In [ ]:
layEncInt = Dense(dimInter, activation='relu')
layMean = Dense(dimEncode)
laySigma = Dense(dimEncode)
layDecInt = Dense(dimInter, activation='relu')
layDecode = Dense(dimInput, activation='sigmoid')

inputs = Input(shape=(dimInput,))
interEncode = layEncInt(inputs)
zMean = layMean(interEncode)
zSigmaLog = laySigma(interEncode) # log for linear dense

def sampling(args):
    zMean, zSigmaLog = args
    epsilon = K.random_normal(shape=(K.shape(zMean)[0], dimEncode),
                              mean=0., stddev=stdEps)
    return zMean + K.exp(zSigmaLog) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
# z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
z = Lambda(sampling)([zMean, zSigmaLog])
encoder = Model(inputs, z)

inputZ = Input(shape=(dimEncode,))
interDecode = layDecInt(inputZ)
decode = layDecode(interDecode)
decoder = Model(inputZ, decode)

## Construct the autoencoder

In [ ]:
vae = Model(inputs, decoder(encoder(inputs)))
vae.summary()

## Compile and train

In [ ]:
def lossVAE(zMean, zSigmaLog):
    def loss(tensorInput, tensorDecode):
        lossRecon =  metrics.binary_crossentropy(tensorInput, tensorDecode)
        lossKL = - 0.5 * K.sum(1 + 2 * zSigmaLog - K.square(zMean) - K.square(K.exp(zSigmaLog)), axis=-1)
#         lossKL = - 0.5 * K.mean(1 + zSigmaLog - K.square(zMean) - K.exp(zSigmaLog), axis=-1)
        return ratRecon * lossRecon + (1 - ratRecon) * lossKL
    return loss


vae.compile(optimizer=nameOptim, loss=lossVAE(zMean, zSigmaLog))
tic = time()
history = vae.fit(xTrain, xTrain,
                epochs=numEpochs,
                batch_size=sizeBatch,
                shuffle=True,
                validation_data=(xTest, xTest))
timeTrain = time() - tic

## Show the historical training progress 

In [ ]:
print("traing consumed: " + str(timeTrain) + " seconds")
plotProgress(history)

## Get the encoding and decoding results of testing data, and get the mean/std of the encoding 

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encodeTest = encoder.predict(xTest)
decodeTest = decoder.predict(encodeTest)
meanEncTest = np.mean(encodeTest, axis=0)
stdEncTest = np.std(encodeTest, axis=0)

## Compare original digitals with the decoding results

In [ ]:
plotCompDecode(xTest, decodeTest, sizeDigit = (28, 20))

## Plot the scatter of the encoding space

In [ ]:
xlim = (meanEncTest[0] - 4*stdEncTest[0], meanEncTest[0] + 4*stdEncTest[0])
ylim = (meanEncTest[1] - 4*stdEncTest[1], meanEncTest[1] + 4*stdEncTest[1])

plotScatterEncode(encodeTest, None, xlim, ylim, numShow=10000)
# scoreSilh = silhouette_score(encodeTest, yTest)  

## Plot the decoding results from the encoding scatter 

In [ ]:
# display a 2D manifold of the digits
plotScatterDecode(decoder, (28,20), [-2,2], [-2,2])

In [ ]:
print(timeTrain, history.history["loss"][numEpochs-1], history.history["val_loss"][numEpochs-1])